In [ ]:
# Instructions for submission

# 1. Rename this file to groupXX_PPGHR.ipynb where XX is your group number as visible in the Google spreadsheet
# 2. State the team members (e-mail, legi):
# example@student.ethz.ch, XX-YYY-ZZZ
# TO BE FILLED
# TO BE FILLED
# 3. Kaggle team name: TO BE FILLED
# 4. Upload this file in a zipped folder together with your final predictions to the provided Polybox link. See the Submission section in the PDF for more details.

In [ ]:
# Import necessary libraries
import matplotlib.dates as mdates
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy as sp
from scipy.signal import find_peaks, peak_prominences, savgol_filter
from scipy.signal import butter, sosfilt


In [ ]:
sampling_rate = 128  # Hz

# Load data item containing the PPG, HR, and IMU signals from all phases
data = np.load('/kaggle/input/24-exercise1/mhealth24_data_public.npy', allow_pickle=True).item() # now it is a dict
#print(data)
print('Keys for data:', data.keys())

# Example to extract the data from phase 0
phase0_data = data['phase 0']
print('Keys for phase 0:', phase0_data.keys())

# Get the individual signals from phase 0
ppg_phase0 = phase0_data['PPG wrist']
ref_hr_phase0 = phase0_data['ground truth HR']  # only available for phase 0, 2, and 4 (training data)
IMU_X_phase0 = phase0_data['IMU X wrist']
IMU_Y_phase0 = phase0_data['IMU Y wrist']
IMU_Z_phase0 = phase0_data['IMU Z wrist']

In [ ]:
sig = data["phase 2"]["PPG head"]
spectrum = np.fft.fft(sig)
freq = np.fft.fftfreq(len(sig))
plt.plot(freq, abs(sig))

In [ ]:
## Windows Configurations
winSize = 8*sampling_rate # Ground truth BPM provided in 8 second windows
winShift = 3*sampling_rate # Successive ground truth windows overlap by 3 seconds

# CHOOSE WINDOW IN FILE
eval_window_idx = 3

offset = eval_window_idx*winShift

window_start = offset
window_end = winSize+offset
offset += winShift

#print(len(sig),len(IMU_X_phase0_filtered_smooth), len(IMU_Y_phase0_filtered_smooth), len(IMU_Z_phase0_filtered_smooth))

print(f"Win start,end: {window_start}, {window_end}")
ppg_window = sig[window_start:window_end]

accx_window = IMU_X_phase0[window_start:window_end]
accy_window = IMU_Y_phase0[window_start:window_end]
accz_window = IMU_Z_phase0[window_start:window_end]
print(len(ppg_window), len(accx_window), len(accy_window), len(accz_window))

In [ ]:
# Function to plot any signal with time on the x-axis
def plot_signal(signal, title, ylabel, plot_window_start, sampling_rate=128, peaks = []):
    x = np.linspace(plot_window_start, plot_window_start+(len(signal) / sampling_rate), len(signal))
    t = pd.to_datetime(x, unit='s')

    fig, ax = plt.subplots(figsize=(15,5))
    ax.plot(t, signal)
    
    if len(peaks)>0:
        for p in peaks:
            plt.axvline(p, alpha = 0.2)
            
    ax.set_title(title)
    ax.set_xlabel('Time [min:sec]')
    ax.set_ylabel(ylabel)
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%M:%S'))
    
    plt.show()
    
def plot_signal_wrapper(signal, title,ylabel, peaks = []):
    plot_window_start = 3  # in seconds
    plot_window_end = 30  # in seconds
    
    #convert peaks for plotting
    if len(peaks)>0:
        peaks = [p/sampling_rate for p in peaks]
        peaks = [p for p in peaks if plot_window_start<= p <= plot_window_end]
        peaks = pd.to_datetime(peaks, unit='s')
    
    #plot raw
    plot_signal(signal[plot_window_start*sampling_rate:plot_window_end*sampling_rate], title,ylabel, peaks = peaks, plot_window_start = plot_window_start)

# If you want to be able to interactively look at your plotted data (e.g., zooming in or out),
# uncomment the line with "%matplotlib widget" below
# Careful: This does not work on Kaggle, but requires that you run the Jupyter Notebook locally on your computer
# If you have an interactive plot and you want to go back to the non-interactive plot, comment the line with 
# "%matplotlib widget" out and restart your kernel
# If you accidently run this script on Kaggle when "%matplotlib widget" is not commented out and you receive an error afterwards that your plot cannot be plotted, comment "%matplotlib widget" out and restart the kernel via "Run << Factory reset"

# %matplotlib widget

# Example plot of a 10-second window of the PPG signal
#plot_window_start = 20  # in seconds
#plot_window_end = 35  # in seconds
#plot_signal(sig_filtered[plot_window_start*sampling_rate:plot_window_end*sampling_rate], 'PPG wrist', 'Amplitude', add_peaks = True)

In [ ]:
def butter_bandpass(lowcut, highcut, fs, order):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    sos = butter(order, [low, high], btype="band",output="sos")
    return sos
def butter_bandpass_filt(data, lowcut, highcut,fs, order):
    sos = butter_bandpass(lowcut, highcut, fs,order)
    y = sosfilt(sos, data)
    return y

def BandpassFilter(signal, fs):
    lo, hi = 40/60, 180/60
    b, a = sp.signal.butter(3, (lo, hi), btype='bandpass', fs=fs)
    return sp.signal.filtfilt(b, a, signal)

def FreqTransform(x, freqs, low_freqs, fft_len):
    norm_x = (x - np.mean(x))/(max(x)-min(x))
    fft_x = np.fft.rfft(norm_x, fft_len)
    mag_freq_x = np.abs(fft_x)[low_freqs]
    return mag_freq_x, fft_x

def CalcConfidence(chosen_freq, freqs, fft_ppg):
    win = (40/60.0)
    win_freqs = (freqs >= chosen_freq - win) & (freqs <= chosen_freq + win)
    abs_fft_ppg = np.abs(fft_ppg)
    conf_val = np.sum(abs_fft_ppg[win_freqs])/np.sum(abs_fft_ppg)

    return conf_val


In [ ]:
def get_hr_2(sig, accx, accy, accz, title="", ylabel="", sampling_rate=128, plot=False, just_estimate=False, verbose=False):    
    if not just_estimate:
        hr_estimate = get_hr_2(sig, accx, accy, accz, title=title, ylabel=ylabel, sampling_rate=sampling_rate, plot=plot, just_estimate=True, verbose=verbose)
       
    #plot raw
    if plot:
        plot_signal_wrapper(sig, title,ylabel)
 
    #filter
    

 
    #sig_filtered = butter_bandpass_filt(sig, 40/60, 180/60, sampling_rate, 3)
    ppg_bandpass = BandpassFilter(sig, fs=128)
    accx_bandpass = accx#BandpassFilter(accx, fs=sampling_rate)
    accy_bandpass = accy#BandpassFilter(accy, fs=sampling_rate)
    accz_bandpass = accz#BandpassFilter(accz, fs=sampling_rate)
    
    # Aggregate accelerometer data into single signal
    accy_centered = accy_bandpass - np.mean(accy_bandpass)  # Centering accy_bandpass
    acc_mag_unfiltered = np.sqrt(accx_bandpass**2 + accy_centered**2 + accz_bandpass**2)
    acc_mag = BandpassFilter(acc_mag_unfiltered, fs=sampling_rate)
    peaks = find_peaks(ppg_bandpass, height = 10, distance=35)[0]
    
    if verbose:
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15,4))
        ax1.title.set_text('Signal with Time Domain FindPeaks()')
        ax1.plot(sig_filtered)
        ax1.plot(peaks, sig_filtered[peaks], "x")

        ax2.title.set_text('Aggregated Accelerometer Data')
        ax2.plot(acc_mag, color="purple")
        plt.show()
    # Use FFT length larger than the input signal size for higher spectral resolution.
    fft_len=len(sig_filtered)*4
 
    # Create an array of frequency bins
    freqs = np.fft.rfftfreq(fft_len, 1 / sampling_rate) # bins of width 0.12207031
 
    # The frequencies between 40 BPM and 180 BPM Hz
    low_freqs = (freqs >= (40/60)) & (freqs <= (180/60))
    
    mag_freq_ppg, fft_ppg = FreqTransform(sig_filtered, freqs, low_freqs, fft_len)
    
    mag_freq_acc, fft_acc = FreqTransform(acc_mag, freqs, low_freqs, fft_len)
    
    peaks_ppg = find_peaks(mag_freq_ppg, height=30, distance=1)[0]
    peaks_acc = find_peaks(mag_freq_acc, height=30, distance=1)[0]
    
    # Sort peaks in order of peak magnitude
    sorted_freq_peaks_ppg = sorted(peaks_ppg, key=lambda i:mag_freq_ppg[i], reverse=True)
    sorted_freq_peaks_acc = sorted(peaks_acc, key=lambda i:mag_freq_acc[i], reverse=True)
    
    # Use the frequency peak with the highest magnitude, unless the peak is also present in the accelerometer peaks.
    use_peak = sorted_freq_peaks_ppg[0]
    for i in range(len(sorted_freq_peaks_ppg)):
        # Check nearest two peaks also
        cond1 = sorted_freq_peaks_ppg[i] in sorted_freq_peaks_acc
        cond2 = sorted_freq_peaks_ppg[i]-1 in sorted_freq_peaks_acc
        cond3 = sorted_freq_peaks_ppg[i]+1 in sorted_freq_peaks_acc
        
        if cond1 or cond2 or cond3:
            continue
        else:
            use_peak = sorted_freq_peaks_ppg[i]
            break  
            

 
        
    chosen_freq = freqs[low_freqs][use_peak]
    prediction = chosen_freq * 60
    confidence = CalcConfidence(chosen_freq, freqs, fft_ppg)
    if plot:
        plot_signal_wrapper(ppg_bandpass, title,ylabel, peaks = peaks)
    if verbose:
        plt.title("PPG Frequency Magnitude")
        plt.plot(mag_freq_ppg)
        plt.plot(peaks_ppg, mag_freq_ppg[peaks_ppg], "x")
        plt.show()
        
        plt.title("ACC Frequency Magnitude")
        plt.plot(mag_freq_acc, color="purple")
        plt.plot(peaks_acc, mag_freq_acc[peaks_acc], "x")
        plt.show()
        
        print("PPG Freq Peaks: ", peaks_ppg)
        print("ACC Freq Peaks: ", peaks_acc)
        
        print("PPG Freq Peaks Sorted: ", sorted_freq_peaks_ppg)
        print("ACC Freq Peaks Sorted: ", sorted_freq_peaks_acc)
        print("Use peak: ", use_peak)
        print(f"Predicted BPM: {prediction}, {chosen_freq} (Hz), Confidence: {confidence}")        
        
    
    # Sum frequency spectrum near pulse rate estimate and divide by sum of entire spectrum

    #prediction, confidence = 60, 0.9  # Placeholder values
    return prediction, confidence         
        
    #find peaks
    #from scipy.signal import find_peaks, peak_prominences
    #peaks = find_peaks(sig_filtered, distance = 3, prominence = 10)[0]
    #prominences = peak_prominences(sig_filtered, peaks)[0]
    #min_prominence = np.quantile(prominences, 0.2)
    
    #if not just_estimate:
    #    #make min_prominence dependent on heartrate
    #    hr_factor = (hr_estimate-40)/(180-40)
    #    min_prominence = np.quantile(prominences, 0.15-0.1*hr_factor)
 
    ##recompute peaks
    #peaks = find_peaks(sig_filtered, prominence = min_prominence, distance=sampling_rate/(180/60))[0]
    
            
    ##plot with peaks
    #if plot:
    #    plot_signal_wrapper(sig_filtered, title,ylabel, peaks = peaks)
    
    ## extract median time between peaks
    #time_between_peaks = []
    #for i in range(1, len(peaks)):
    #    time_between_peaks.append((peaks[i]-peaks[i-1])/sampling_rate)
    ##print(time_between_peaks[:10])
    
    
    ## zscore normalize
    #outlier_low = np.mean(time_between_peaks) - 1.2 * np.std(time_between_peaks)
    #outlier_high = np.mean(time_between_peaks) + 1.2 * np.std(time_between_peaks)
    #len_before = len(time_between_peaks)
    #time_between_peaks = [p for p in time_between_peaks if outlier_low <= p <= outlier_high]   
    #print(f"removed {len_before-len(time_between_peaks)} outliers out of {len_before} datapoints")
    
    #return 60/np.mean(time_between_peaks)

In [ ]:
#before hr

def get_hr_1(sig,title="",ylabel="", sampling_rate = 128, plot = False, just_estimate = False):
    
    
    if not just_estimate:
        hr_estimate = get_hr_1(sig,title="",ylabel="", sampling_rate = 128, plot = False, just_estimate = True)
    
    
    #plot raw
    if plot:
        plot_signal_wrapper(sig, title,ylabel)

    #filter

    sig_filtered = butter_bandpass_filt(sig, 40/60, 180/60, sampling_rate, 3)
    
    
    #find peaks
    from scipy.signal import find_peaks, peak_prominences
    peaks = find_peaks(sig_filtered, distance = 3, prominence = 10)[0]
    prominences = peak_prominences(sig_filtered, peaks)[0]
    min_prominence = np.quantile(prominences, 0.2)
    
    if not just_estimate:
        #make min_prominence dependent on heartrate
        hr_factor = (hr_estimate-40)/(180-40)
        min_prominence = np.quantile(prominences, 0.15-0.1*hr_factor)

    #recompute peaks
    peaks = find_peaks(sig_filtered, prominence = min_prominence, distance=sampling_rate/(180/60))[0]
    
            
    #plot with peaks
    if plot:
        plot_signal_wrapper(sig_filtered, title,ylabel, peaks = peaks)
    
    # extract median time between peaks
    time_between_peaks = []
    for i in range(1, len(peaks)):
        time_between_peaks.append((peaks[i]-peaks[i-1])/sampling_rate)
    #print(time_between_peaks[:10])
    
    
    # zscore normalize
    outlier_low = np.mean(time_between_peaks) - 1.2 * np.std(time_between_peaks)
    outlier_high = np.mean(time_between_peaks) + 1.2 * np.std(time_between_peaks)
    len_before = len(time_between_peaks)
    time_between_peaks = [p for p in time_between_peaks if outlier_low <= p <= outlier_high]   
    #print(f"removed {len_before-len(time_between_peaks)} outliers out of {len_before} datapoints")
    
    return 60/np.mean(time_between_peaks)

In [ ]:
accx_window_filter=BandpassFilter(accx_window, fs=sampling_rate)
accy_window_filter=BandpassFilter(accy_window, fs=sampling_rate)
accz_window_filter=BandpassFilter(accz_window, fs=sampling_rate)
sig_filtered = butter_bandpass_filt(sig, 40/60, 180/60, sampling_rate, 3)

hr = get_hr_2(sig,accx_window,accy_window,accz_window, plot = True)
#hr = get_hr_2(sig_filtered,accx_window_filter,accy_window_filter,accz_window_filter, plot = True)

pred = get_hr_1(sig,title = f"phase {phase}", ylabel="signal", plot = True)


print(hr)

In [ ]:
## Define Kalman filter parameters
#Q = 1e-5  # Process noise covariance
#R = 0.1  # Measurement noise covariance
#x_initial = 0  # Initial state
#P_initial = 1  # Initial covariance

## Define the Kalman filter function
#def kalman_filter(data, Q, R, x_initial, P_initial):
#    n = len(data)
#    x_hat = np.zeros(n)  # Estimated state
#    P = np.zeros(n)  # Covariance
#    K = np.zeros(n)  # Kalman gain

#    x_hat_minus = x_initial
#    P_minus = P_initial

#    for i in range(n):
#        # Prediction update
#        x_hat_minus = x_hat[i-1]
#        P_minus = P[i-1] + Q

#        # Measurement update
#        K[i] = P_minus / (P_minus + R)
#        x_hat[i] = x_hat_minus + K[i] * (data[i] - x_hat_minus)
#        P[i] = (1 - K[i]) * P_minus

#    return x_hat

## Apply the Kalman filter to IMU data
#IMU_X_phase0_filtered = kalman_filter(IMU_X_phase0, Q, R, x_initial, P_initial)
#IMU_Y_phase0_filtered = kalman_filter(IMU_Y_phase0, Q, R, x_initial, P_initial)
#IMU_Z_phase0_filtered = kalman_filter(IMU_Z_phase0, Q, R, x_initial, P_initial)

## Apply additional noise reduction using Savitzky-Golay filter
#window_length = 21
#polyorder = 3
#IMU_X_phase0_filtered_smooth = savgol_filter(IMU_X_phase0_filtered, window_length, polyorder)
#IMU_Y_phase0_filtered_smooth = savgol_filter(IMU_Y_phase0_filtered, window_length, polyorder)
#IMU_Z_phase0_filtered_smooth = savgol_filter(IMU_Z_phase0_filtered, window_length, polyorder)
#print(IMU_X_phase0_filtered_smooth)
## Plotting
#plt.figure(figsize=(12, 6))

#plt.subplot(3, 1, 1)
#plt.plot(IMU_X_phase0_filtered_smooth, label='IMU X (filtered)')
#plt.title('Filtered IMU X Data - Phase 0')
#plt.xlabel('Sample')
#plt.ylabel('Acceleration')
#plt.legend()
#plt.grid(True)

#plt.subplot(3, 1, 2)
#plt.plot(IMU_Y_phase0_filtered_smooth, label='IMU Y (filtered)')
#plt.title('Filtered IMU Y Data - Phase 0')
#plt.xlabel('Sample')
#plt.ylabel('Acceleration')
#plt.legend()
#plt.grid(True)

#plt.subplot(3, 1, 3)
#plt.plot(IMU_Z_phase0_filtered_smooth, label='IMU Z (filtered)')
#plt.title('Filtered IMU Z Data - Phase 0')
#plt.xlabel('Sample')
#plt.ylabel('Acceleration')
#plt.legend()
#plt.grid(True)

#plt.tight_layout()
#plt.show()



In [ ]:
# Function to print the mean and median absolute error between your predicted HR and the reference HR
# With this function, you can evaluate the resulting score that you would obtain on the public dataset
# with your predicted HR values on Kaggle
def print_score(pred_hr, ref_hr):
    err = np.abs(np.asarray(pred_hr) - np.asarray(ref_hr))
    print("Mean error: {:4.3f}, Median error {:4.3f}".format(np.mean(err), np.median(err)))
    print("Resulting score {:4.3f}".format(0.5 * np.mean(err) + 0.5 * np.median(err)))
    return 0.5 * np.mean(err) + 0.5 * np.median(err)

# Example on how to use the print_score function with randomly generated HR values as the predictions
pred_hr_phase0 = list(np.random.randint(40, 180, len(ref_hr_phase0)))
print_score(pred_hr_phase0, ref_hr_phase0)

In [ ]:
## Windows Configurations
winSize = 8*sampling_rate # Ground truth BPM provided in 8 second windows
winShift = 3*sampling_rate # Successive ground truth windows overlap by 3 seconds

# CHOOSE WINDOW IN FILE
eval_window_idx = 3

offset = eval_window_idx*winShift

window_start = offset
window_end = winSize+offset
offset += winShift

#print(len(sig),len(IMU_X_phase0_filtered_smooth), len(IMU_Y_phase0_filtered_smooth), len(IMU_Z_phase0_filtered_smooth))

print(f"Win start,end: {window_start}, {window_end}")
ppg_window = sig[window_start:window_end]

accx_window = IMU_X_phase0[window_start:window_end]
accy_window = IMU_Y_phase0[window_start:window_end]
accz_window = IMU_Z_phase0[window_start:window_end]

In [ ]:
def chunker(seq, size):
    return [seq[pos:pos + size] for pos in range(0, len(seq), size)]
    
dat = {
    0:data["phase 0"]["PPG wrist"],
    2:data["phase 2"]["PPG head"],
    4:data["phase 4"]["PPG head"]
}
scores = []
for phase in [0,2,4]:
    
    sig = dat[phase]
    true_hr = data[f"phase {phase}"]['ground truth HR']
    
    
    plot_this = False
    collected_preds = []
    
    c = 0
    ccritical = 139
    #not
    """
    for section in chunker(sig, 30*sampling_rate):   
        
        if c==ccritical:
            plot_this = True
        print(len(section), len(chunker(sig, 30*sampling_rate)))
        pred = get_hr_1(section,title = f"not pau phase {phase}", ylabel="signal", plot = plot_this)
        #pred_pau, confidence = get_hr_2(section,accx_window,accy_window,accz_window, plot = True)
        if plot_this:
            print("pred",pred)
        if c==ccritical:
            plot_this = False
        collected_preds.append(pred)
        c+=1
    """
    #this
    for section in range(len(chunker(sig, 30*sampling_rate))):   
        offset = abs(section*winShift)
        window_start = offset
        window_end = winSize+offset
        offset += winShift
        print(f"Win start,end: {window_start}, {window_end}")
        ppg_window = sig[window_start:window_end]

        accx_window = IMU_X_phase0[window_start:window_end]
        accy_window = IMU_Y_phase0[window_start:window_end]
        accz_window = IMU_Z_phase0[window_start:window_end]
        if c==ccritical:
            plot_this = True
        #print(len(section))
        #pred = get_hr_1(section,title = f"not pau phase {phase}", ylabel="signal", plot = plot_this)
        pau, confidence = get_hr_2(ppg_window,accx_window,accy_window,accz_window, plot = True)
        if plot_this:
            print("pred",pred)
        if c==ccritical:
            plot_this = False
        collected_preds.append(pred)
        c+=1
        
    scores.append(print_score(collected_preds, true_hr))
    
    print(f"\ni\t\tpredicted\ttrue\t\tdiff")
    diffs = []
    for i in range(len(collected_preds)):
        diff = round(collected_preds[i]-true_hr[i],1)
        diffs.append(diff)
        print(f"{i}\t\t{round(collected_preds[i],1)}\t\t{round(true_hr[i],1)}\t\t{diff}")
        break
    print(f"avg diff: {np.mean(diffs)}")
print(np.mean(scores))


In [ ]:
data["phase 5"].keys()

In [ ]:
# For each phase, you should now have obtained a list of predicted HR values
# Below, we give an example of how you can produce the submission.csv file from your predicted HR values
# To demonstrate the format of the submission.csv file, we provide an example with randomly generated HR values
# For phase 0, 1, 2, and 3 you should each obtain 396 HR values
# For phase 4 and 5 you should each obtain 57 HR values
# IMPORTANT: You have to replace the following predicted HR values with your predicted HR values!

collected_preds = []
dat = {
    0:data["phase 0"]["PPG wrist"],
    1:data["phase 1"]["PPG head"],
    2:data["phase 2"]["PPG head"],
    3:data["phase 3"]["PPG wrist"],
    4:data["phase 4"]["PPG head"],
    5:data["phase 5"]["PPG head"]
}
for phase in [0,1,2,3,4,5]:

    
    sig = dat[phase]    
    collected_preds_per_phase = []

    for section in chunker(sig, 30*sampling_rate):        
        pred = get_hr_1(section,title = f"phase {phase}", ylabel="signal")
        collected_preds_per_phase.append(pred)
    collected_preds.append(collected_preds_per_phase)

pred_hr_phase0 = collected_preds[0]
pred_hr_phase1 = collected_preds[1]
pred_hr_phase2 = collected_preds[2]
pred_hr_phase3 = collected_preds[3]
pred_hr_phase4 = collected_preds[4]
pred_hr_phase5 = collected_preds[5]

# You can keep the below code unchanged to produce the submission.csv file
pred_hr_phases = [pred_hr_phase0, pred_hr_phase1, pred_hr_phase2,
                  pred_hr_phase3, pred_hr_phase4, pred_hr_phase5]
ids = []
pred_hr_flattened = []


for phase_counter in range(len(pred_hr_phases)):
    for hr_counter in range(len(pred_hr_phases[phase_counter])):
        pred_hr_flattened.append(pred_hr_phases[phase_counter][hr_counter])
        ids.append(f'phase{phase_counter}_{hr_counter}')

# If you use Kaggle, on the right side in tab "Output", you should now see a file called "submission.csv" after pressing "refresh"
# Download the file and submit it to the competition on Kaggle to obtain a score on the leaderboard for your team
df = pd.DataFrame({'Id': ids, 'Predicted': pred_hr_flattened})
df.to_csv('/kaggle/working/submission.csv', index=False)

In [ ]:
#def write_new_csv():
#    collected_preds = []
#    for phase in [0,1,2,3,4,5]:
#        data_phase = data[f'phase {phase}']
#        ppg_phase0 = data_phase['PPG wrist']
#        ref_hr_phase0 = data_phase['ground truth HR']  # only available for phase 0, 2, and 4 (training data)
#        IMU_X_phase0 = data_phase['IMU X wrist']
#        IMU_Y_phase0 = data_phase['IMU Y wrist']
#        IMU_Z_phase0 = data_phase['IMU Z wrist']
        
#    df = pd.DataFrame({'Id': ids, 'Predicted': pred_hr_flattened})
#    df.to_csv('/kaggle/working/submission.csv', index=False)